# Sistema Multi-Agente de Consulta de Normativas Aeronáuticas

## Descripción

En este ejercicio, diseñarás un **sistema multi-agente que orquesta consultas a diferentes normativas aeronáuticas**. Un agente orquestador decidirá qué agentes especializados invocar según la consulta del usuario, y siempre verificará con la normativa EASA antes de dar una respuesta final.

### Arquitectura
```
Consulta del usuario
    ↓
[Agente Orquestador]
    ├→ [EASA] → Siempre consultado (obligatorio)
    ├→ [DEF-STAN] → Consultado si se menciona
    ├→ [EDA] → Consultado si se menciona
    ├→ [FAA] → Consultado si se menciona
    └→ [JSSG] → Consultado si se menciona
    ↓
Respuesta integral verificada con EASA
```

### Agentes Especializados Disponibles
- **DEF-STAN**: Normativa DEF STAN
- **EDA**: Normativa EDA (European Defence Agency)
- **FAA**: Normativa FAA (Federal Aviation Administration)
- **JSSG**: Normativa JSSG (Joint Service Specification Guide)
- **EASA**: Normativa EASA (European Union Aviation Safety Agency) - **Verificación obligatoria**

### Conceptos Clave
- **Connected Agent Tools**: Permite que el orquestador invoque agentes especializados
- **Arquitectura Multi-agente**: Consulta inteligente basada en el contexto de la pregunta
- **Verificación EASA**: Todas las respuestas deben ser verificadas con la normativa EASA

### Cargar librerías y variables de entorno

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ConnectedAgentTool, MessageRole, ListSortOrder
from azure.identity import DefaultAzureCredential

load_dotenv(find_dotenv(usecwd=True))
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")

### Conectar al Cliente de Agentes (Agent Client)

In [ ]:
print("Connecting to Azure AI Agents Client...")
agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True,
    )
)
print("Agents client connected successfully")

### Configurar Agentes Especializados Existentes

Los agentes especializados ya están creados con conocimiento exclusivo de diferentes normativas. Configuramos las herramientas para conectar con ellos.

In [ ]:
# IDs of already created specialized agents
AGENT_IDS = {
    "DEF-STAN": "asst_rkE8wccEYNW8VMkyYvGbCJGA",
    "EDA": "asst_fOW4towcWWIuHUN7Q8cBI05x",
    "FAA": "asst_acdzvNNQe6951L1v1wnIWRJJ",
    "JSSG": "asst_8w48R9lpBhAP5DluxYpYzMjE",
    "EASA": "asst_4Fh3Cj4SBLhjWmgeQLgXTzjV"
}

print("Configuring tools for specialized agents...")
print(f"\nAvailable agents:")
for name, agent_id in AGENT_IDS.items():
    print(f"  - {name}: {agent_id}")


### Crear Herramientas de Agentes Conectados (Connected Agent Tools)

Creamos herramientas para que el agente orquestador pueda invocar a cada uno de los agentes especializados.

In [ ]:
print("Creating connected agent tools...")

# Tool for DEF-STAN
defstan_tool = ConnectedAgentTool(
    id=AGENT_IDS["DEF-STAN"],
    name="DEF_STAN",
    description="Consulta la normativa DEF STAN. Usa esta herramienta cuando el usuario mencione específicamente DEF STAN o estándares de defensa británicos."
)

# Tool for EDA
eda_tool = ConnectedAgentTool(
    id=AGENT_IDS["EDA"],
    name="EDA",
    description="Consulta la normativa EDA (European Defence Agency). Usa esta herramienta cuando el usuario mencione específicamente EDA o normativa de defensa europea."
)

# Tool for FAA
faa_tool = ConnectedAgentTool(
    id=AGENT_IDS["FAA"],
    name="FAA",
    description="Consulta la normativa FAA (Federal Aviation Administration). Usa esta herramienta cuando el usuario mencione específicamente FAA o regulaciones estadounidenses."
)

# Tool for JSSG
jssg_tool = ConnectedAgentTool(
    id=AGENT_IDS["JSSG"],
    name="JSSG",
    description="Consulta la normativa JSSG (Joint Service Specification Guide). Usa esta herramienta cuando el usuario mencione específicamente JSSG o especificaciones militares."
)

# Tool for EASA (MANDATORY)
easa_tool = ConnectedAgentTool(
    id=AGENT_IDS["EASA"],
    name="EASA",
    description="Consulta la normativa EASA (European Union Aviation Safety Agency). ESTA HERRAMIENTA DEBE SER SIEMPRE CONSULTADA antes de proporcionar una respuesta final al usuario."
)

print("Connected agent tools created:")
print("  - DEF-STAN")
print("  - EDA")
print("  - FAA")
print("  - JSSG")
print("  - EASA (mandatory verification)")


### Crear el Agente Orquestador

Este agente analizará la consulta del usuario, decidirá qué agentes especializados invocar, y siempre verificará la respuesta con EASA.

In [ ]:
orchestrator_agent_name = "normativas-orchestrator"
orchestrator_agent_instructions = """
Eres un agente orquestador especializado en normativas aeronáuticas. Tu función es:

1. Análisis de la Consulta: 
   - Analiza cuidadosamente la pregunta del usuario para identificar qué normativas específicas se mencionan o se requieren.
   
2. Selección de Agentes:
   - Si el usuario menciona DEF-STAN, DEF STAN o estándares de defensa británicos: consulta al agente DEF-STAN
   - Si el usuario menciona EDA o defensa europea: consulta al agente EDA
   - Si el usuario menciona FAA o regulaciones estadounidenses: consulta al agente FAA
   - Si el usuario menciona JSSG o especificaciones militares: consulta al agente JSSG
   
3. Verificación OBLIGATORIA con EASA:
   - SIEMPRE debes consultar al agente EASA y dar información sobre la pregunta del usuario en base a la normativa EASA.

4. Respuesta Estructurada:
   - Proporciona una respuesta clara y estructurada
   - Indica claramente qué normativas has consultado
   - Si hay conflictos entre normativas, señálalos claramente
   
5. Principios:
   - Sé preciso y técnico en tus respuestas
   - Cita las fuentes (normativas) consultadas
   - Si no estás seguro, indícalo claramente
   - Prioriza la seguridad y el cumplimiento normativo

Recuerda: La consulta a EASA es OBLIGATORIA.
"""

print("Creando agente orquestador...")
orchestrator_agent = agents_client.create_agent(
    model=model_deployment,
    name=orchestrator_agent_name,
    instructions=orchestrator_agent_instructions,
    tools=[
        defstan_tool.definitions[0],
        eda_tool.definitions[0],
        faa_tool.definitions[0],
        jssg_tool.definitions[0],
        easa_tool.definitions[0],
    ],
)
print(f"Agente orquestador creado (id: {orchestrator_agent.id})")
print(f"\nEl agente tiene acceso a {len(orchestrator_agent.tools)} herramientas:")
print("  - DEF-STAN")
print("  - EDA")
print("  - FAA")
print("  - JSSG")
print("  - EASA (verificación obligatoria)")

### Realizar Consultas al Sistema Multi-Agente

Introduce tu consulta sobre normativas aeronáuticas en el `prompt`. El agente orquestador decidirá qué agentes consultar y siempre verificará con EASA.

In [ ]:
import requests
import json

print("Creating conversation thread...")
thread = agents_client.threads.create()
print(f"Thread created (id: {thread.id})")

# Enter your query here
prompt = "What are the overspeed success criteria for each normative?"

print(f"\n{'='*80}")
print(f"USER QUERY:")
print(f"{'='*80}")
print(f"{prompt}")
print(f"{'='*80}\n")

print("Sending query to orchestrator agent...")
message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=prompt,
)

print("Processing query... The orchestrator agent is consulting the necessary regulations.")

run = agents_client.runs.create_and_process(
    thread_id=thread.id, 
    agent_id=orchestrator_agent.id
)
print(f"run.id: {run.id}")
print(f"Processing completed with status: {run.status}\n")

if run.status == "failed":
    print(f"Execution error: {run.last_error}")
else:
    print(f"{'='*80}")
    print(f"AGENT RESPONSE:")
    print(f"{'='*80}\n")
    
    messages = agents_client.messages.list(
        thread_id=thread.id, 
        order=ListSortOrder.ASCENDING
    )
    
    for message in messages:
        if message.text_messages:
            for text_msg in message.text_messages:
                if message.role != MessageRole.USER:
                    print(f"\n{text_msg.text.value}\n")
    
    print(f"{'='*80}")



### Trazar las herramientas usadas

Mediante una petición al URL del endpoint de nuestro proyecto + thread + run, obtenemos todos los subprocesos y herramientas usadas.

In [ ]:

# Make REST request to get run steps
print(f"\n{'='*80}")
print(f"OBTAINING RUN EXECUTION STEPS (REST API)")
print(f"{'='*80}\n")

# Build endpoint URL
api_version = "2024-07-01-preview"
steps_url = f"{project_endpoint}/threads/{thread.id}/runs/{run.id}/steps?api-version=v1"

# Get authentication token
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential(
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
)
# Get token for the correct Azure AI scope
token = credential.get_token("https://ai.azure.com/.default")
# Make GET request
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

response = requests.get(steps_url, headers=headers)

if response.status_code == 200:
    steps_data = response.json()
    print("ANALYSIS OF TOOLS USED")
    print("="*80)
    
    # Get and sort steps by creation timestamp (chronological order)
    all_steps = steps_data.get("data", [])
    sorted_steps = sorted(all_steps, key=lambda x: x.get("created_at", 0))
    
    # Filter only tool_calls type steps
    tool_calls_found = False
    steps_n = 0
    for step in sorted_steps:
        if step.get("type") == "tool_calls":
            tool_calls_found = True
            step_details = step.get("step_details", {})
            tool_calls = step_details.get("tool_calls", [])
            steps_n += 1
            for idx, tool_call in enumerate(tool_calls, 1):
                print(f"\n[TOOL CALL #{steps_n}]")
                
                if tool_call.get('type') == 'connected_agent':
                    connected = tool_call.get('connected_agent', {})
                    print(f"AGENT: {connected.get('name')}")
                    print(f"ARGUMENT (PROMPT):")
                    print(f"  {connected.get('arguments')}")
                    print(f"RESPONSE (first 100 characters):")
                    output = connected.get('output', '')
                    print(f"  {output[:100]}...")
                
                print(f"\nExecution time: {step.get('completed_at') - step.get('created_at')} seconds")
                print("-" * 80)
            
    
    if not tool_calls_found:
        print("\nNo tool calls found in this run.")
    
else:
    print(f"Request error: {response.status_code}")
    print(f"Response: {response.text}")

